In [24]:
import sqlite3.dbapi2 as sqlite
from statistics import *
import folium

In [2]:
conn=sqlite.connect('adresy.sqlite')
curs=conn.cursor()

In [3]:
curs.execute("CREATE TABLE IF NOT EXISTS obce(id_obce INTEGER PRIMARY KEY, meno_obce TEXT);")

In [4]:
curs.execute("CREATE TABLE IF NOT EXISTS ulice(id_ulice INTEGER PRIMARY KEY, meno_ulice TEXT, id_obce INTEGER, FOREIGN KEY(id_obce) REFERENCES obca(id_obce));")

In [5]:
curs.execute("CREATE TABLE IF NOT EXISTS cisla(id_domu INTEGER PRIMARY KEY, cislo_domu TEXT, lat FLOAT, lon FLOAT, id_ulice INTEGER, FOREIGN KEY(id_ulice) REFERENCES ulice(id_ulice));")

In [6]:
f = open('obce.csv', 'r')
for riadok in f:
    oid, omeno = riadok[:-1].split(',')
    try:
        curs.execute("INSERT INTO obce(id_obce, meno_obce) VALUES(?, ?)", (int(oid), omeno))
    except sqlite.IntegrityError:
        pass
f.close()
conn.commit()

In [7]:
f = open('ulice.csv', 'r')
for riadok in f:
    uid, umeno, oid=riadok[:-1].split(',')
    try:
        curs.execute("INSERT INTO ulice(id_ulice, meno_ulice, id_obce) VALUES(?, ?, ?)", (int(uid), umeno, int(oid)))
    except sqlite.IntegrityError:
        pass
f.close()
conn.commit()

In [8]:
f = open('cisla.csv', 'r')
for riadok in f:
    did, cislo, la, lo, uid=riadok[:-1].split(',')
    try:
        curs.execute("INSERT INTO cisla(id_domu, cislo_domu, lat, lon, id_ulice) VALUES (?, ?, ?, ?, ?)", (int(did), cislo, la, lo, int(uid)))
    except sqlite.IntegrityError:
        pass
f.close()
conn.commit()

In [9]:
# Zobraz vsetkych ulic v Ziline
curs.execute("SELECT u.meno_ulice FROM ulice u, obce o WHERE o.meno_obce='Žilina' AND o.id_obce=u.id_obce")
zoznam = curs.fetchall()
zoznam[:10]

[('Lumumbova',),
 ('Nerudova',),
 ('Rauchova',),
 ('Komsomolcov',),
 ('Ostrovského',),
 ('Urxova',),
 ('Suche Bátora',),
 ('Togliattiho',),
 ('1.mája',),
 ('Republiky',)]

In [10]:
# Pocet vsetkych ulic v Bratislave
curs.execute("SELECT COUNT(u.meno_ulice) FROM ulice u, obce o WHERE o.meno_obce LIKE 'Bratislava%' AND o.id_obce=u.id_obce")
curs.fetchall()

[(1967,)]

In [11]:
# Pocet cisel na ulici Univerzitna v Ziline
curs.execute("SELECT COUNT(c.id_domu) FROM ulice u, obce o, cisla c WHERE o.meno_obce='Žilina' AND o.id_obce=u.id_obce AND u.meno_ulice='Univerzitná' AND c.id_ulice=u.id_ulice")
curs.fetchall()

[(8,)]

In [12]:
# Zoznam cisel na ulici Univerzitna v Ziline
curs.execute("SELECT c.cislo_domu, c.lat, c.lon FROM ulice u, obce o, cisla c WHERE o.meno_obce='Žilina' AND o.id_obce=u.id_obce AND u.meno_ulice='Univerzitná' AND c.id_ulice=u.id_ulice")
curs.fetchall()

[('2', 49.2032503, 18.7540002),
 ('1', 49.2032990766, 18.7552876829),
 ('25', 49.2097410367, 18.7538823558),
 ('23', 49.209669971, 18.7542788524),
 ('6A', 49.2038165412, 18.7540302348),
 ('3A', 49.2017855, 18.7540148),
 ('3B', 49.2020801, 18.7542544),
 ('3A', 49.2017855, 18.7540148)]

In [13]:
def lokalizacia_ulice(mesto, ulica, mesto_like = False):
    if mesto_like:
        curs.execute("SELECT c.cislo_domu, c.lat, c.lon FROM ulice u, obce o, cisla c WHERE o.meno_obce LIKE ? AND o.id_obce=u.id_obce AND u.meno_ulice=? AND c.id_ulice=u.id_ulice", (mesto+"%", ulica))
    else:
        curs.execute("SELECT c.cislo_domu, c.lat, c.lon FROM ulice u, obce o, cisla c WHERE o.meno_obce=? AND o.id_obce=u.id_obce AND u.meno_ulice=? AND c.id_ulice=u.id_ulice", (mesto, ulica))
    zoznam = curs.fetchall()
    return zoznam

In [14]:
lokalizacia_ulice("Bratislava", "Vavilovova", True)

[('7', 48.1276667968, 17.1134491629),
 ('5', 48.1273703434, 17.1134176997),
 ('2', 48.126365066, 17.1112832053),
 ('6', 48.126336447, 17.1120965562),
 ('4', 48.1263504629, 17.1117006112),
 ('8', 48.1263250417, 17.1124707368),
 ('10', 48.1263141695, 17.1127624083),
 ('12', 48.1263115999, 17.1130538546),
 ('14', 48.1262894235, 17.1133477899),
 ('16', 48.126286003, 17.1136428507),
 ('26', 48.1271581883, 17.1151716642),
 ('24', 48.1271718568, 17.1148307198),
 ('22', 48.1271869334, 17.1144804038),
 ('20', 48.1271957338, 17.1141279817),
 ('1', 48.1267458854, 17.1133717639),
 ('3', 48.127027152, 17.1133902597),
 ('18', 48.1260145662, 17.1135282254)]

In [15]:
def lokalizacia_adresy(mesto, ulica, cislo, mesto_like=False):
    if mesto_like:
        curs.execute("SELECT c.lat, c.lon FROM ulice u, obce o, cisla c WHERE o.meno_obce LIKE ? AND o.id_obce=u.id_obce AND u.meno_ulice=? AND c.id_ulice=u.id_ulice AND c.cislo_domu=?", (mesto+"%", ulica, cislo))
    else:
        curs.execute("SELECT c.lat, c.lon FROM ulice u, obce o, cisla c WHERE o.meno_obce=? AND o.id_obce=u.id_obce AND u.meno_ulice=? AND c.id_ulice=u.id_ulice AND c.cislo_domu=?", (mesto, ulica,cislo))
    zoznam = curs.fetchall()
    return zoznam

In [16]:
lokalizacia_adresy("Žilina", "Univerzitná", 1)

[(49.2032990766, 18.7552876829)]

In [17]:
ulica = "Univerzitná"
mesto = "Žilina"

In [18]:
r = lokalizacia_ulice(mesto, ulica)
r

[('2', 49.2032503, 18.7540002),
 ('1', 49.2032990766, 18.7552876829),
 ('25', 49.2097410367, 18.7538823558),
 ('23', 49.209669971, 18.7542788524),
 ('6A', 49.2038165412, 18.7540302348),
 ('3A', 49.2017855, 18.7540148),
 ('3B', 49.2020801, 18.7542544),
 ('3A', 49.2017855, 18.7540148)]

In [19]:
C, Lat, Lon = zip(*r)
C

('2', '1', '25', '23', '6A', '3A', '3B', '3A')

In [20]:
sur = zip(Lat, Lon)
[i for i in sur]

[(49.2032503, 18.7540002),
 (49.2032990766, 18.7552876829),
 (49.2097410367, 18.7538823558),
 (49.209669971, 18.7542788524),
 (49.2038165412, 18.7540302348),
 (49.2017855, 18.7540148),
 (49.2020801, 18.7542544),
 (49.2017855, 18.7540148)]

In [88]:
C, Lat, Lon = zip(*r)
sur = zip(Lat, Lon)
Lat_s, Lon_s = mean(Lat), mean(Lon)
ul_map = folium.Map(location=(Lat_s, Lon_s),zoom_start = 170)
for p, c in zip(sur, C):
    ul_map.add_child(folium.CircleMarker(p,popup ="%s,%s" %(ulica, c), radius=6, color='red', fill_color='red', fill_opacity=0.3))
ul_map

### Mapa psov
zobrazit na mape, kde zije zadane plemeno<br>
zobrazit na mape psov zijuch na zadanej ulici

In [36]:
import pandas as pd
df = pd.read_json('PsyPresov.json')
psi = df[['Plemeno', 'Ulica_(chovu_psa)', 'Orient._č.']].rename(columns={'Ulica_(chovu_psa)' : 'Ulica', 'Orient._č.' : 'Cislo'})
psi = psi[psi['Ulica'] != '']
psi

,Plemeno,Ulica,Cislo
0,Maltézky psík,Agátová,3
1,Bišon,Belianska,6
2,Yorkshirský teriér,Belianska,6
3,Nemecký ovčiak,Belianska,7
4,Kríženec,Belianska,11
...,...,...,...
3184,Pudel,Sovia,7
3185,Pudel,Sovia,7
3186,Čivava,Sovia,29
3187,Kríženec,Teriakovská,6


In [41]:
def zadaj_Plemeno(pl):
    return psi[psi['Plemeno'] == pl]
    

In [48]:
pl = psi[psi['Plemeno'] == 'bígel']
bydlisko = pl[['Ulica', 'Cislo']]
bydlisko

,Ulica,Cislo
282,Arm. gen. Svobodu,22
388,Bratislavská,8
1160,Majakovského,15
2255,Veselá,5
2411,Wolkerova,11
2792,Lomnická,16
3041,Švábska,60


In [84]:
U = list(bydlisko['Ulica'].values)
C = [c.strip() for c in bydlisko['Cislo'].values]
adr = zip(U,C)
adresy=[i for i in adr]
sur = []
for u,c in adresy:
    gps = lokalizacia_adresy("Prešov", u, c)
    if gps == []:
        gps = [None]
    sur += gps
sur


[(48.9857735617, 21.2656540658),
 (48.988680833, 21.2671800005),
 (49.0074069229, 21.2367935938),
 None,
 (49.0046655364, 21.2375464815),
 (48.9725905741, 21.2624909971),
 (48.972341275, 21.2593929411)]

In [87]:
osur = [i for i in sur if i != None]
Lat, Lon = zip(*osur)
Lat_s, Lon_s = mean(Lat), mean(Lon)
ul_map = folium.Map(location=(Lat_s,Lon_s),zoom_start = 14)
for p, a in zip(sur, adresy):
    if p != None:
        ul_map.add_child(folium.CircleMarker(p,popup ="%s,%s" %(a[0],a[1]), radius=6, color='red', fill_color='red', fill_opacity=0.3))
ul_map